In [1]:
from requests_html import HTMLSession

session = HTMLSession()
portatil_links = []

for i in range(1,3):
    print(f'Pagina {i}')
    r = session.get(url=f"https://www.coolmod.com/portatiles-portatiles/?pagina=1{i}")
    
    articles = r.html.find('article')
    
    print(articles)

Pagina 1
[<Element 'article' class=('product-card', 'card', 'card-compact', 'shadow-[0_2px_8px_0px_rgb(0,0,0,0.1)]', 'rounded-lg', 'product', 'bg-base-100', 'w-full', 'min-h-[495px]') data-code='PROD-036493'>, <Element 'article' class=('product-card', 'card', 'card-compact', 'shadow-[0_2px_8px_0px_rgb(0,0,0,0.1)]', 'rounded-lg', 'product', 'bg-base-100', 'w-full', 'min-h-[495px]') data-code='PROD-036076'>, <Element 'article' class=('product-card', 'card', 'card-compact', 'shadow-[0_2px_8px_0px_rgb(0,0,0,0.1)]', 'rounded-lg', 'product', 'bg-base-100', 'w-full', 'min-h-[495px]') data-code='PROD-034985'>, <Element 'article' class=('product-card', 'card', 'card-compact', 'shadow-[0_2px_8px_0px_rgb(0,0,0,0.1)]', 'rounded-lg', 'product', 'bg-base-100', 'w-full', 'min-h-[495px]') data-code='PROD-033127'>, <Element 'article' class=('product-card', 'card', 'card-compact', 'shadow-[0_2px_8px_0px_rgb(0,0,0,0.1)]', 'rounded-lg', 'product', 'bg-base-100', 'w-full', 'min-h-[495px]') data-code='PROD-

In [ ]:
import json
from requests_html import HTMLSession

session = HTMLSession()
info_list = []

for link in portatil_links:
    r = session.get(f'https://www.mediamarkt.es{link}')
    tables = r.html.find('td')
    
    info = {}
    for i in range(0, len(tables), 2):
        clave = tables[i].text.strip()
        valor = tables[i+1].text.strip()
        info[clave] = valor
    
    info_list.append(info)

with open('info_list.json', 'w', encoding='utf-8') as f:
    json.dump(info_list, f, ensure_ascii=False, indent=4)

for info in info_list:
    print(info)

In [ ]:
<div class="collapse-content prose" bis_skin_checked="1">
                                    <p>Diseño</p>

<ul>
	<li>Tipo de producto - Portátil</li>
	<li>Color del producto - Negro</li>
	<li>Factor de forma - Concha</li>
	<li>Año de introducción - 2023</li>
</ul>

<p>Exhibición</p>

<ul>
	<li>Diagonal de la pantalla - 39,6 cm (15.6")</li>
	<li>Resolución de la pantalla - 1920 x 1080 Pixeles</li>
	<li>Pantalla táctil - No</li>
	<li>Tipo HD - Full HD</li>
	<li>Máxima velocidad de actualización - 144 Hz</li>
</ul>

<p>Procesador</p>

<ul>
	<li>Fabricante de procesador - Intel</li>
	<li>Generación del procesador - Intel® Core™ i5 de 13ma Generación</li>
	<li>Familia de procesador - Intel® Core™ i5</li>
	<li>Modelo del procesador - i5-13500H</li>
	<li>Frecuencia del procesador turbo - 4,7 GHz</li>
	<li>Frecuencia base de núcleo de rendimiento - 4,7 GHz</li>
	<li>Frecuencia turbo máxima del núcleo de eficiencia - 3,5 GHz</li>
	<li>Número de núcleos de procesador - 12</li>
	<li>Núcleos de rendimiento - 4</li>
	<li>Núcleos de eficiencia - 8</li>
	<li>Caché del procesador - 18 MB</li>
	<li>Tipo de cache en procesador - Smart Cache</li>
	<li>Potencia base del procesador - 45 W</li>
	<li>Potencia turbo máxima - 95 W</li>
</ul>

<p>Memoria</p>

<ul>
	<li>Memoria interna - 16 GB</li>
	<li>Tipo de memoria interna - DDR5-SDRAM</li>
	<li>Velocidad de memoria del reloj - 4800 MHz</li>
	<li>Memoria interna máxima - 64 GB</li>
</ul>

<p>Medios de almacenaje</p>

<ul>
	<li>Capacidad total de almacenaje - 512 GB</li>
	<li>Unidad de almacenamiento - SSD</li>
	<li>Capacidad total de SSD - 512 GB</li>
	<li>Número de unidades SSD instalados - 1</li>
	<li>SDD, capacidad - 512 GB</li>
	<li>Interfaces del SDD - PCI Express 4.0</li>
	<li>Factor de forma de disco SSD - M.2</li>
	<li>Tipo de unidad óptica - No</li>
	<li>Tarjeta de lectura integrada - Si</li>
	<li>Tarjetas de memoria compatibles - MicroSD (TransFlash)</li>
</ul>

<p>Gráficos</p>

<ul>
	<li>Fabricante de GPU (unidad de procesamiento gráfico) externa - NVIDIA</li>
	<li>Modelo de adaptador de gráficos discretos - NVIDIA GeForce RTX 4060</li>
	<li>Adaptador gráfico incorporado - Si</li>
	<li>Fabricante de GPU (unidad de procesamiento gráfico) integrada - Intel</li>
	<li>Adaptador de gráficos discreto - Si</li>
	<li>Familia de adaptador gráfico incorporado - Intel Iris Xe Graphics</li>
	<li>Modelo de adaptador gráfico incorporado - Intel Iris Xe Graphics</li>
	<li>Capacidad memoria de adaptador gráfico - 8 GB</li>
	<li>Tipo de memoria de gráficos discretos - GDDR6</li>
</ul>

<p>Audio</p>

<ul>
	<li>Sistema de audio - DTS:X Ultra</li>
	<li>Número de altavoces incorporados - 2</li>
	<li>Potencia de altavoz - 2 W</li>
	<li>Micrófono incorporado - Si</li>
</ul>

<p>Cámara fotográfica</p>

<ul>
	<li>Cámara frontal - Si</li>
	<li>Resolución de la cámara frontal (numérica) - 1 MP</li>
	<li>Tipo de cámara frontal HD - HD</li>
</ul>

<p>Conexión</p>

<ul>
	<li>Estándar Wi-Fi - Wi-Fi 6E (802.11ax)</li>
	<li>Wi-Fi estándares - Wi-Fi 6E (802.11ax)</li>
	<li>Conexión a red móvil - No</li>
	<li>Fabricante del controlador WLAN - Intel</li>
	<li>Ethernet - Si</li>
	<li>Ethernet LAN, velocidad de transferencia de datos - 100,1000 Mbit/s</li>
	<li>Bluetooth - Si</li>
	<li>Versión de Bluetooth - 5.2</li>
</ul>

<p>Puertos e Interfaces</p>

<ul>
	<li>Cantidad de puertos USB 2.0 - 1</li>
	<li>Cantidad de puertos tipo A USB 3.2 Gen 1 (3.1 Gen 1) - 1</li>
	<li>Cantidad de puertos tipo C USB 3.2 Gen 2 (3.1 Gen 2) - 2</li>
	<li>Ethernet LAN (RJ-45) cantidad de puertos - 1</li>
	<li>Número de puertos HDMI - 1</li>
	<li>Cantidad de Mini DisplayPorts - 1</li>
	<li>Micrófono, jack de entrada - Si</li>
	<li>Combo de salida de auriculares / micrófono del puerto - Si</li>
	<li>Tipo de puerto de carga - Toma de entrada de CC</li>
</ul>

<p>Teclado</p>

<ul>
	<li>Dispositivo apuntador - Touchpad</li>
	<li>Distribución física - ANSI</li>
	<li>Teclado numérico - Si</li>
	<li>Retroiluminación de teclado - Si</li>
</ul>

<p>Software</p>

<ul>
	<li>Sistema operativo instalado - FreeDOS</li>
</ul>

<p>Batería</p>

<ul>
	<li>Tecnología de batería - Ión de litio</li>
	<li>Capacidad de batería - 54 Wh</li>
</ul>

<p>Control de energía</p>

<ul>
	<li>Potencia de adaptador AC - 150 W</li>
</ul>

<p>Seguridad</p>

<ul>
	<li>Ranura para cable de seguridad - Si</li>
	<li>Tipo de ranura de bloqueo del cable - Kensington</li>
	<li>Módulo de plataforma confiable (TPM) - Si</li>
</ul>

<p>Peso y dimensiones</p>

<ul>
	<li>Ancho - 360 mm</li>
	<li>Profundidad - 238 mm</li>
	<li>Altura - 22,7 mm</li>
	<li>Peso - 2,08 kg</li>
</ul>

                            </div>